In [11]:
#our goal is to have top K artist recommendations for a user, so that they would 
from __future__ import print_function
import pandas as pd
import implicit
import numpy as np
import scipy
import random
from operator import itemgetter
from tqdm import tqdm_notebook as tqdm

df = pd.read_csv('lastfm_9000_users.csv', na_filter=False)
df = df[df.plays > 10] #our threshold of relevance is 10 plays
df = df.drop(['Unnamed: 0'], axis=1)

## TO DO

BIG THINGS: 
- set-up cross-validation
- calculate precision/recall/AUC using ALL user-pairs, not only k-recommended values
- calculate NDCG 
- calculate ATOP 
- calculate speed performance

CASES TO EVALUATE:
- sparse vs. non-sparse data
- threshold of relevance
- which users should we hold out (those with > 10/20/30 values?)

PERFORMANCE:
- speed up split_train_per_user, by figuring out how to access zero values faster
- speed up precision, recall evaluation

In [12]:
#create sparse matrix from dataframe object
def create_sparse_matrix(data, user_user = True):
    """
    Creates sparse matrix (csr_matrix) out of pandas dataframe.
    
    Parameters: 
    - data: Dataframe of user/artist data
    - user_user: determines whether output will be for user-to-user or item-to-item collaborative filtering
                 if user_user is True, then rows will be items and columns will be users
    
    Returns: 
    - plays_sparse: a sparse csr_matrix
    
    """
    print("Creating sparse matrix...")
    #grab unique users/artist IDS
    users = list(np.sort(data.user_id.unique()))
    artists = list(data.artist_name.unique())
    plays = list(data.plays)

    # user-user set-up
    if (user_user == True):
        rows = data.artist_name.astype('category', categories=artists).cat.codes
        cols = data.user_id.astype('category', categories=users).cat.codes
        plays_sparse = scipy.sparse.csr_matrix((plays, (rows, cols)), shape=(len(artists),len(users)))

    #item-item set-up
    else:    
        rows = data.user_id.astype('category', categories=users).cat.codes
        cols = data.artist_name.astype('category', categories=artists).cat.codes
        plays_sparse = scipy.sparse.csr_matrix((plays, (rows, cols)), shape=(len(users),len(artists)))
        
        
        
    return plays_sparse

In [13]:
#calculate how sparse the matrix is
def calculate_sparsity(M):
    """
    Calculates how sparse this matrix
    """
    matrix_size = float(M.shape[0]*M.shape[1]) # Number of possible interactions in the matrix
    num_plays = len(M.nonzero()[0]) # Number of items interacted with
    sparsity = 100*(1 - float(num_plays/matrix_size))
    return sparsity

In [14]:
#split train, test using all user pairs
def make_train_all_user_pairs(data, test_pct):
    """
    params:
        data: data set in csr_matrix format
        test_pct: percentage to be test set
    """
    #create copies of dataset for training and test data
    test = data.copy()
    train = data.copy()
    
    #alter train data, masking/holding-out random user-pair values for some users
    nonzero_idx = train.nonzero() #find indices in data where interaction exists
    
    
    nonzero_pairs = zip(nonzero_idx[0], nonzero_idx[1]) #create pairs of (user, item) index
    
    #determine how many user-pair values we need to mask, according to test_pct
    random.seed(0) #for reproducibility
    num_samples = int(np.ceil(test_pct * len(nonzero_pairs)))
    samples = random.sample(nonzero_pairs, num_samples) #sample random number of user-item pairs without replacement
    print(type(samples))
    #get user, item row and column indices
    user_idx = [index[0] for index in samples] 
    item_idx = [index[1] for index in samples] 
    
    train[user_idx,item_idx] = 0 #mask the randomly chosen user-item pairs
    train.eliminate_zeros() #remove zeros in sparse arrays that was made previously
    
    return train, test, list(set(user_idx)), samples #output unique list of user rows that were altered

In [5]:
#split train, test by user only with k interactions
def split_train_test_per_user(data, test_pct, k = 20):
    """
    Create train matrix with masked values and dictionary of test values 
    
    Parameters:
    - data: csr_matrix, assuming matrix is user-user (item as rows, columns as users)
    - test_pct: percentage of items to mask per user
    
    Output:
    - train: masked matrix
    - test: list of tuples of held out data ((user_idx, item_idx), plays)
    """
    random.seed(0) #for reproducibility
    
    train = data.copy().T.tocsr() #transpose to make procedure easier/more intuitive
    
    test = dict() #dict to keep track of masked user-item values
    
    user_count = 0
    
    #for each user in the training set
    for user_idx in tqdm(range(train.get_shape()[0])):

        #get indices of interactions of this user
        nonzero_idx = train[user_idx].nonzero()
        
        #only hold out users that have enough data (greater than k interactions)
        if nonzero_idx[1].shape[0] >= k:
            user_count += 1
            #create list of tuples: interaction index (row, col) with the number of plays
            nonzero_pairs = [((user_idx, item_idx), train[user_idx,item_idx]) for item_idx in nonzero_idx[1]]
            
            #sort tuples descending by value
            nonzero_sorted = sorted(nonzero_pairs, key = itemgetter(1), reverse = True)

            #get top k values, then sample test_pct% randomly from subset
            top_values = nonzero_sorted[0:k]

            #sample random number of item_indexes without replacement
            num_samples = int(np.ceil(test_pct * k))
            samples = random.sample(top_values, num_samples) 

            #append user_idx, item_
            test[user_idx] = [pair[0][1] for pair in samples]
            
            #mask the randomly chosen items of this user
            for pair in samples:
                train[pair[0][0], pair[0][1]] = 0
    
    #remove zeros in sparse arrays to save space
    train.eliminate_zeros()
    
    return train.T.tocsr(), test, user_count #convert matrix back

In [6]:
#calculate how many interactions are masked compared to previous dataset
def pct_masked(original, altered):
    altered_n = altered.nonzero()[0].shape[0]
    original_n = original.nonzero()[0].shape[0]
    return (original_n - altered_n)/float(altered_n)

In [17]:
#used to evaluate model
def evaluate(model, test, M_rec, n_rec = 20):
    """
    Calculate precision/recall
    
    parameters:
    - model: fitted implicit model that will perform recommendations
    - test: list containing tuples that are heldout for each user
    - M: csr_matrix of item-users, used in fit
    - n_rec: how many recommendations the system outputs
    
    
    returns:
    - two numpy arrays containing precision and recall
    """
    precision= list()
    recall= list()

#     M_rec = M.T.tocsr() #transpose to recommend
    
    print('Evaluating model...')
    #calculate precision/recall for each user, append results to list
    for user, holdout_items in tqdm(test.items()):
        print(user)
        rec = model.recommend(user, M_rec, N=n_rec, filter_already_liked_items=True)
        rec_items = [pair[0] for pair in rec]
        print(holdout_items)
        print(rec_items)
        #count true positives in recommended items
        tp = float(0)
        for item in holdout_items: 
            if item in rec_items:
                tp += 1
        print(tp)
                
        #calculate precision and recall
        precision.append(tp/n_rec) #fraction of recommendations that are in hold-out set
        recall.append(tp/len(holdout_items)) #fraction of hold-out set that were reocmmended
    
    return np.asarray(precision), np.asarray(recall)

In [18]:
#MAIN SCRIPT
plays_sparse = create_sparse_matrix(df)

#filter out users with < 15 artists/reduce sparsity if needed
print('Matrix Sparsity:', calculate_sparsity(plays_sparse))

#split train,test by masking random values by user
train, test, user_count = split_train_test_per_user(plays_sparse, 0.20, 20)
print("Percentage of original data masked:", pct_masked(plays_sparse, train))
print("Users masked:", user_count)

# Matrix Factorization
model = implicit.als.AlternatingLeastSquares(factors=50)

# K-Nearest Neighbors
# model = implicit.nearest_neighbours.BM25Recommender()

# train model 
print("Fitting model...")
model.fit(train, show_progress=True)

precision, recall = evaluate(model, test, plays_sparse)
print('Mean of Precision:',np.mean(precision)*100,'%')
print('Std of Precision:',np.std(precision)*100,'%')
print('Mean of Recall:',np.mean(recall)*100,'%')
print('Std of Recall:',np.std(recall)*100,'%')

Creating sparse matrix...


/Users/TimGimi/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:23: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
/Users/TimGimi/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:24: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead


Matrix Sparsity: 99.9078520973


  0%|          | 0/15 [00:00<?, ?it/s]


Percentage of original data masked: 0.0894432763057
Users masked: 8178
Fitting model...


...

[150, 27, 1244, 165]
[917, 905, 151, 84, 897, 638, 592, 129, 57, 313, 56, 315, 53, 14, 3702, 165, 894, 92, 373, 6822]
1.0
8472
[555, 952, 572, 584]
[620, 1424, 143, 566, 219, 1235, 572, 638, 233, 634, 133, 1668, 1188, 70, 854, 89, 814, 651, 69, 56]
1.0
8473
[25, 1994, 3217, 5447]
[363, 98, 8, 3, 37, 1585, 28, 1416, 7224, 6, 598, 804, 1139, 5479, 384, 8545, 457, 1134, 1601, 779]
0.0
8474
[1544, 258, 2976, 2507]
[2507, 3545, 1714, 846, 28, 3094, 4756, 2843, 2976, 1900, 1544, 172, 2290, 3629, 1147, 150, 363, 3092, 1887, 2542]
3.0
8475
[6790, 431, 256, 5607]
[318, 14, 2480, 33, 6, 907, 1, 3095, 1451, 390, 86, 2250, 458, 878, 155, 5954, 9, 401, 1714, 11]
0.0
8476
[47823, 47825, 47824, 47817]
[662, 10136, 798, 3633, 1114, 702, 14145, 147, 2161, 2413, 794, 4185, 4662, 120, 680, 4902, 2788, 7350, 10119, 26829]
0.0
8477
[2485, 2797, 7058, 18539]
[1906, 233, 1747, 814, 775, 210, 2349, 1234, 2797, 2139, 932, 2517, 566, 56, 213, 1008, 1745, 430, 592, 902]
1.0
8478
[2123, 242, 70, 1002]
[352, 1008,

[48029, 1219, 548, 7435]
[2409, 10234, 3451, 2156, 1067, 4337, 1071, 1678, 801, 1076, 1700, 3904, 556, 1068, 4439, 1086, 106, 8901, 1441, 2477]
0.0
8554
[1095, 6486, 17157, 1982]
[647, 209, 1297, 186, 4356, 434, 2380, 8167, 29, 1190, 1287, 10173, 13385, 1967, 56, 1845, 195, 1959, 7088, 185]
0.0
8555
[25, 18414, 10584, 42564]
[25, 4205, 2390, 2361, 1970, 12081, 5374, 15201, 2362, 2359, 4808, 13132, 4883, 4229, 4295, 462, 2341, 433, 2294, 2876]
1.0
8556
[92, 639, 352, 1259]
[53, 373, 129, 390, 733, 737, 12, 151, 326, 57, 726, 59, 728, 780, 2165, 52, 256, 718, 897, 634]
0.0
8557
[347, 8125, 676, 1699]
[1890, 7371, 347, 2319, 184, 5042, 3324, 666, 3079, 3686, 1130, 1299, 10136, 848, 3270, 17, 4449, 2109, 568, 8215]
1.0
8558
[184, 17, 47, 4439]
[221, 1008, 47, 198, 184, 775, 568, 2419, 862, 1301, 518, 5457, 3274, 2006, 224, 770, 1295, 777, 313, 8571]
2.0
8559
[1416, 332, 4668, 129]
[917, 214, 1353, 151, 200, 341, 186, 591, 43, 897, 594, 92, 2100, 70, 1234, 980, 2258, 1506, 90, 1147]
0.0
856

[1148, 70, 2937, 1471]
[592, 3629, 4809, 2848, 383, 905, 1293, 85, 2189, 165, 92, 1262, 554, 2113, 1301, 917, 2843, 2123, 1745, 1424]
0.0
8642
[172, 3, 2, 3305]
[41, 172, 457, 2258, 33, 164, 1266, 901, 401, 1714, 1967, 49, 899, 28, 56, 150, 458, 617, 34, 120]
1.0
8643
[121, 12211, 56, 2415]
[2232, 894, 256, 3495, 373, 249, 129, 5616, 7122, 56, 4120, 357, 371, 1870, 4121, 150, 592, 771, 5467, 4117]
1.0
8644
[7664, 1886, 570, 1039]
[47, 1130, 713, 651, 150, 22, 5147, 1147, 990, 14, 1714, 1086, 1647, 1301, 3270, 9, 566, 20, 548, 369]
0.0
8645
[12483, 908, 23056, 973]
[373, 5534, 1309, 3213, 3999, 2, 592, 2895, 27, 7504, 365, 226, 826, 454, 1000, 3508, 357, 4214, 2529, 250]
0.0
8646
[19202, 15711, 34416, 3091]
[725, 6177, 5600, 368, 4726, 2479, 3477, 1305, 405, 1293, 810, 1506, 150, 300, 22, 2890, 370, 34, 2880, 27]
0.0
8647
[3173, 2171, 6613, 1265]
[3334, 812, 6608, 1397, 2171, 3173, 1230, 3349, 210, 352, 2179, 216, 1265, 743, 3411, 1240, 255, 1241, 3134, 1259]
3.0
8648
[3820, 264, 373, 2

[4305, 1265, 1388, 1432]
[1388, 2179, 1240, 310, 1945, 2939, 1265, 1008, 352, 3637, 911, 1259, 2171, 3349, 3334, 215, 1241, 1913, 737, 2937]
2.0
8727
[12010, 851, 850, 722]
[363, 1967, 172, 1147, 1278, 41, 84, 186, 3, 1763, 1273, 313, 5697, 1797, 256, 634, 1287, 897, 86, 911]
0.0
8730
[3753, 31146, 1970, 1967]
[1967, 1668, 854, 1076, 588, 2052, 101, 562, 625, 3478, 804, 13, 3753, 186, 636, 2984, 3471, 4415, 56, 638]
2.0
8731
[3697, 48508, 3, 6522]
[315, 897, 2895, 390, 357, 341, 1519, 2479, 79, 1211, 1147, 49, 594, 1305, 1370, 3604, 417, 12, 2977, 2131]
0.0
8732
[625, 7292, 572, 576]
[2975, 854, 7296, 2984, 5466, 572, 143, 576, 2747, 2755, 2680, 2993, 69, 3482, 638, 574, 2757, 9927, 3423, 946]
2.0
8733
[216, 348, 830, 7376]
[120, 814, 911, 230, 397, 210, 371, 129, 69, 810, 70, 727, 256, 731, 1853, 106, 1670, 365, 1008, 126]
0.0
8735
[428, 326, 33, 18]
[9, 369, 33, 862, 848, 20, 1506, 1295, 878, 1370, 276, 2250, 17, 47, 326, 2258, 2279, 192, 634, 14]
2.0
8736
[4507, 669, 553, 5346]
[47,

[42218, 1235, 965, 12938]
[949, 132, 1803, 7765, 1662, 6320, 8089, 7759, 7754, 7173, 3569, 7187, 11352, 4538, 4313, 3720, 1791, 3652, 4320, 960]
0.0
8826
[75, 2857, 3629, 848]
[334, 3100, 1144, 214, 426, 2909, 41, 1991, 594, 899, 3629, 2132, 638, 980, 200, 3606, 1967, 1508, 383, 742]
1.0
8827
[2670, 7249, 17008, 2977]
[79, 679, 89, 2977, 1970, 2664, 1508, 370, 15288, 1378, 4656, 1293, 1523, 590, 1967, 1297, 17002, 1391, 378, 760]
1.0
8828
[5853, 15404, 3802, 3796]
[3796, 3382, 3361, 4235, 1131, 4237, 3210, 11510, 4500, 5668, 3384, 5347, 4254, 5863, 9779, 5862, 3802, 5865, 13534, 5613]
2.0
8829
[1481, 214, 18778, 242]
[326, 1234, 394, 350, 313, 1521, 762, 3603, 332, 390, 2097, 341, 913, 50, 796, 397, 2977, 53, 150, 1897]
0.0
8830
[1241, 3137, 1706, 1900]
[1008, 3469, 1900, 2989, 352, 1512, 1544, 722, 1521, 6821, 1302, 3544, 219, 3096, 262, 332, 913, 363, 3100, 394]
1.0
8831
[9157, 1309, 371, 1266]
[725, 368, 1714, 1705, 249, 172, 250, 300, 150, 258, 298, 22, 405, 804, 1378, 846, 1728, 1

In [ ]:
#AUC https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc
#Primary metric: ROC-based
#calculate Precision, Recall, TPr, FPr, AUC by comparing resulting matrix to test_data
#Secondary metric: DCG

#cross_validate(data, k)
#get k-fold indices on train (mask again)
#for each different k-fold, loop through the indices, masked as train, test as not
#train model
#calculate Precision, Recall, AUC, append result to list
#return list of scores for each fold

#calculate ATOP? DCG? 